In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df= pd.read_csv("HR_CLEAN_DATA.csv", index_col=0)
pd.set_option('display.max_columns', None)


* COLUMNAS WORK LIFE BALANCE Y TOTAL WORKING YEARS

In [3]:
#GESTIONO NULOS COLUMNA WORK LIFE BALANCE
#Primero, quito espacios de sobra 
df["Work_Life_Balance"]= df["Work_Life_Balance"].astype(str).str.replace(" ", "").astype("Int64")
df["Work_Life_Balance"].unique()

<IntegerArray>
[3, 99999, 2, 4, 1]
Length: 5, dtype: Int64

In [4]:
#Cambio los valores sentinelas por nan, los nulos de numpy, en las 2 columnas que gestiono Work_Life_Balance y "Total_Working_Years"
df[["Work_Life_Balance", "Total_Working_Years"]]= df[["Work_Life_Balance", "Total_Working_Years"]].replace(99999, np.nan)

#Compruebo que esta columna tiene muy pocos nulos, 98 de 1477 registros
df["Work_Life_Balance"].isnull().sum()

#Total_Working_Years la gestiono más abajo

98

In [5]:
#Miro sus principales estadísticos, comprobando que la desviación estándar es pequeña
df["Work_Life_Balance"].describe().T

count      1379.0
mean     2.757796
std      0.706133
min           1.0
25%           2.0
50%           3.0
75%           3.0
max           4.0
Name: Work_Life_Balance, dtype: Float64

In [6]:
#Decido completar los nulos con la mediana, pero antes miro qué valor me da
df["Work_Life_Balance"].median()

3.0

In [7]:
#Relleno nulos con fillna y la mediana que es 3.
df['Work_Life_Balance'] = df['Work_Life_Balance'].fillna(df["Work_Life_Balance"].median())

In [8]:
#Columna sin nulos recién sacada del horno.
df['Work_Life_Balance'].unique()

<IntegerArray>
[3, 2, 4, 1]
Length: 4, dtype: Int64

In [9]:
#GESTIONO NULOS DE LA COLUMNA TOTAL WORKING YEARS
#primero miro cuántos nulos tengo, aquí son mas: 485
df["Total_Working_Years"].isnull().sum()

485

In [10]:
#y estos son sus values unicos
df["Total_Working_Years"].unique()

array([nan, 34., 22., 28., 20., 21., 33., 40., 18., 25., 15., 17., 26.,
       16., 24., 14., 23., 27., 19., 11., 38., 12., 13., 29., 10., 37.,
       36., 35.,  9., 31., 32.,  8.,  7., 30.,  6.,  5.,  4.,  3.,  2.,
        1.,  0.])

In [11]:
#Miro estadísticos de media y mediana
df["Total_Working_Years"].describe()[["mean", "50%"]]

mean    11.263105
50%     10.000000
Name: Total_Working_Years, dtype: float64

In [12]:
# como la media y la mediana no son muy distintos, uso simpleImputer 
imputer_años_currando = SimpleImputer(strategy = "mean")
años_currando_imputed = imputer_años_currando.fit_transform(df[["Total_Working_Years"]])

años_currando_imputed

array([[11.26310484],
       [34.        ],
       [22.        ],
       ...,
       [ 9.        ],
       [12.        ],
       [11.26310484]])

In [13]:
# Creo nueva columna para comparar resultado con la anterior columna
df["working_years_SimpleImp"] = años_currando_imputed

# por último comprobamos si tenemos nulos. 

In [14]:
#compruebo que no tengo nulos
df['working_years_SimpleImp'].isnull().sum()

0

In [15]:
#comparo las columnas, parece que tiene sentido (ver 1ª y última)
df.iloc[:, 26::2]

,Stock_Option_Level,Training_Times_Last_Year,Years_At_Company,Years_With_Curr_Manager,Remote_Work
Employee_Number,,,,,
162,0,5,20,15,True
259,1,5,33,9,True
319,0,3,22,15,True
2069,2,2,20,6,False
2070,1,5,19,8,False
...,...,...,...,...,...
967,1,6,16,7,False
972,0,5,9,8,True
2465,0,3,8,7,True


In [16]:
#Ahora que he comprobado todo, borro columna antigua
df.drop("Total_Working_Years", axis= 1, inplace=True)

In [17]:
#Renombro la nueva para que se llame como la antigua
df.rename(columns= {"working_years_SimpleImp": "Total_Working_Years"} , inplace=True )

In [18]:
#redondeo para tener datos más precisos, que perderé al convertir a INT
df["Total_Working_Years"] = df["Total_Working_Years"].round(2)

In [19]:
#Finalmente convierto a INT
df["Total_Working_Years"]= df["Total_Working_Years"].astype(int)

In [169]:
df["Total_Working_Years"].unique()

array([14, 11, 10, 17,  5, 19, 26, 20,  7,  3,  1, 15, 24,  9, 21, 12,  6,
       13, 25,  4,  0, 16, 29, 22, 30, 23,  2,  8, 35, 18, 40, 38, 31, 33,
       27, 37, 36, 28, 34, 32])

* MARITAL_STATUS    